In [1]:
from gensim.models import KeyedVectors
import pandas as pd
import numpy as np
from recipe_loader import recipe_loader

# load ingredient vectors
wv = KeyedVectors.load("recipe2vec.wordvectors")

In [12]:
def recipe_vectorizer(recipe: str) -> np.ndarray:
    
    ingredient_vecs = np.array([wv[ingredient] for ingredient in recipe])
    
    sum = np.sum(ingredient_vecs, axis=0)
    
    # recipe_vec = sum / np.linalg.norm(sum)
    recipe_vec = np.mean(ingredient_vecs,axis=0)
    
    return recipe_vec
    

Load the recipes:

In [3]:
recipes = recipe_loader()

In [13]:
recipe_vectors = recipes["Ingredients"].apply(recipe_vectorizer)

In [14]:
from annoy import AnnoyIndex

vec_size = 100

index = AnnoyIndex(vec_size, "angular")

for i, vec in enumerate(recipe_vectors.values):
    index.add_item(i, vec)
    
index.build(10)
index.save(f"recipe2vecIndex.ann")

True

In [6]:
recipes.head()

,RecipeId,Name,AuthorId,AuthorName,CookTime,PrepTime,TotalTime,DatePublished,Description,Images,...,CholesterolContent,SodiumContent,CarbohydrateContent,FiberContent,SugarContent,ProteinContent,RecipeServings,RecipeYield,RecipeInstructions,Ingredients
0,38,Low-Fat Berry Blue Frozen Dessert,1533,Dancer,PT24H,PT45M,PT24H45M,1999-08-09T21:46:00Z,Make and share this Low-Fat Berry Blue Frozen ...,"c(""https://img.sndimg.com/food/image/upload/w_...",...,8.0,29.8,37.1,3.6,30.2,3.2,4.0,NaN,"c(""Toss 2 cups berries with sugar."", ""Let stan...","(blueberries, granulated sugar, vanilla yogurt..."
1,39,Biryani,1567,elly9812,PT25M,PT4H,PT4H25M,1999-08-29T13:12:00Z,Make and share this Biryani recipe from Food.com.,"c(""https://img.sndimg.com/food/image/upload/w_...",...,372.8,368.4,84.4,9.0,20.4,63.4,6.0,NaN,"c(""Soak saffron in warm milk for 5 minutes and...","(saffron, milk, hot green chili peppers, onion..."
2,40,Best Lemonade,1566,Stephen Little,PT5M,PT30M,PT35M,1999-09-05T19:52:00Z,This is from one of my first Good House Keepi...,"c(""https://img.sndimg.com/food/image/upload/w_...",...,0.0,1.8,81.5,0.4,77.2,0.3,4.0,NaN,"c(""Into a 1 quart Jar with tight fitting lid, ...","(sugar, lemons, rind of, lemon, zest of, fresh..."
3,41,Carina's Tofu-Vegetable Kebabs,1586,Cyclopz,PT20M,PT24H,PT24H20M,1999-09-03T14:54:00Z,This dish is best prepared a day in advance to...,"c(""https://img.sndimg.com/food/image/upload/w_...",...,0.0,1558.6,64.2,17.3,32.1,29.3,2.0,4 kebabs,"c(""Drain the tofu, carefully squeezing out exc...","(extra firm tofu, eggplant, zucchini, mushroom..."
4,42,Cabbage Soup,1538,Duckie067,PT30M,PT20M,PT50M,1999-09-19T06:19:00Z,Make and share this Cabbage Soup recipe from F...,"""https://img.sndimg.com/food/image/upload/w_55...",...,0.0,959.3,25.1,4.8,17.7,4.3,4.0,NaN,"c(""Mix everything together and bring to a boil...","(plain tomato juice, cabbage, onion, carrots, ..."


In [15]:
search_query= recipe_vectorizer(["pasta","heavy cream"])

In [32]:
recipes.iloc[150672]["RecipeIngredientParts"]

'("olive oil", "heavy cream", "half-and-half", "parmesan cheese", "fettuccine", "linguine")'

In [30]:
indexes = index.get_nns_by_vector(search_query, 10)
print(indexes)
recipes.iloc[indexes]

[331585, 500759, 150672, 195083, 430407, 93626, 253470, 479639, 325852, 123263]


,RecipeId,Name,AuthorId,AuthorName,CookTime,PrepTime,TotalTime,DatePublished,Description,Images,...,CholesterolContent,SodiumContent,CarbohydrateContent,FiberContent,SugarContent,ProteinContent,RecipeServings,RecipeYield,RecipeInstructions,Ingredients
339054,351745,Alfredo Noodles,1052065,Bay Laurel,PT8M,PT5M,PT13M,2009-01-25T23:53:00Z,I love alfredo and wanted an easy recipe that ...,"c(""https://img.sndimg.com/food/image/upload/w_...",...,82.3,287.0,43.9,1.8,1.2,13.0,8.0,NaN,"""Boil pasta in 4-6 quarts of water on stove to...","(pasta, butter, heavy cream, parmesan cheese)"
511456,529984,My Alfredo,238779,Ted8332,PT10M,PT15M,PT25M,2017-01-18T17:46:00Z,Make and share this My Alfredo recipe from Foo...,"""https://img.sndimg.com/food/image/upload/w_55...",...,99.8,618.4,59.0,2.4,2.3,21.7,6.0,NaN,"c(""Melt the butter in a saucepan over medium h...","(butter, heavy cream, parmesan cheese, sea sal..."
153656,161016,Roasted Vegetables With Fettuccine,9717,carolinafan,PT20M,PT10M,PT30M,2006-03-22T14:37:00Z,I love this recipe because of the different co...,character(0),...,94.1,119.0,41.7,1.9,1.2,11.1,4.0,NaN,"c(""Preheat oven to 450."", ""In 13x9-inch baking...","(olive oil, heavy cream, half-and-half, parmes..."
199023,207723,Fettuccini Alfredo,351910,Cameronsmommy,PT30M,PT10M,PT40M,2007-01-26T14:12:00Z,Make and share this Fettuccini Alfredo recipe ...,"""https://img.sndimg.com/food/image/upload/w_55...",...,255.9,284.8,84.5,0.0,0.2,23.3,NaN,NaN,"c(""Place the butter in a shallow serving dish ...","(fettuccine pasta, unsalted butter, heavy crea..."
439768,456028,Pasta With Walnuts and Vegetable Sauce,560491,AZPARZYCH,PT15M,PT30M,PT45M,2011-05-13T22:44:00Z,Found online; posting for ZWT 7-South America(...,character(0),...,143.9,262.9,108.7,17.0,4.7,19.0,NaN,NaN,"c(""Bring a large pot of salted water to a boil...","(butter, walnuts, fresh corn, frozen corn, cre..."
95456,100886,3 Step Fettuccine Alfredo,56680,Dawnab,PT20M,PT5M,PT25M,2004-09-29T19:59:00Z,Got this off the package of fettuccine noodles...,"""https://img.sndimg.com/food/image/upload/w_55...",...,115.5,265.0,31.7,1.4,0.9,10.4,8.0,NaN,"c(""Pour butter over cooked noodles and stir."",...","(fettuccine, butter, parmesan cheese, heavy cr..."
258953,269372,Ham and Pea Pasta,324390,cookiedog,PT15M,PT15M,PT30M,2007-12-02T23:48:00Z,Make and share this Ham and Pea Pasta recipe f...,character(0),...,252.4,1264.3,171.8,9.8,7.9,47.4,2.0,NaN,"c(""Brown butter in a large skillet."", ""Add pep...","(butter, crushed red pepper flakes, ham, heavy..."
489896,507955,Fettuccine With Cream Prosciutto and Peas,283251,dicentra,PT10M,PT10M,PT20M,2013-10-17T19:55:00Z,Make and share this Fettuccine With Cream Pros...,character(0),...,246.7,357.2,71.4,2.2,2.6,21.8,4.0,NaN,"c(""In a large frying pan, melt butter over med...","(butter, heavy cream, peas, prosciutto, parmes..."
333184,345703,Fettuccine All'alfredo,461834,diner524,PT20M,PT0S,PT20M,2008-12-30T01:05:00Z,I found this recipe in my &quot;Four Ingredien...,"c(""https://img.sndimg.com/food/image/upload/w_...",...,189.0,401.1,49.0,0.0,0.2,17.4,4.0,NaN,"c(""Melt butter in a large pan. Add the cream ...","(butter, heavy cream, parmesan cheese, fresh f..."
125672,132090,Penne With Creamy Vodka Sauce,185105,Stacky5,PT15M,PT20M,PT35M,2005-08-02T18:50:00Z,"Oh, YUM! I got this from Campbell's Fall Fami...","c(""https://img.sndimg.com/food/image/upload/w_...",...,39.6,1333.0,88.9,9.4,21.9,14.1,NaN,NaN,"c(""Prepare the pasta according to the package ...","(penne pasta, vodka, fresh basil, crushed red ..."


In [ ]:
indexes = index.get_nns_by_item(0, 10)

recipes.iloc[indexes]

,RecipeId,Name,AuthorId,AuthorName,CookTime,PrepTime,TotalTime,DatePublished,Description,Images,...,CholesterolContent,SodiumContent,CarbohydrateContent,FiberContent,SugarContent,ProteinContent,RecipeServings,RecipeYield,RecipeInstructions,Ingredients
0,38,Low-Fat Berry Blue Frozen Dessert,1533,Dancer,PT24H,PT45M,PT24H45M,1999-08-09T21:46:00Z,Make and share this Low-Fat Berry Blue Frozen ...,"c(""https://img.sndimg.com/food/image/upload/w_...",...,8.0,29.8,37.1,3.6,30.2,3.2,4.0,NaN,"c(""Toss 2 cups berries with sugar."", ""Let stan...","(blueberries, granulated sugar, vanilla yogurt..."
430950,446923,Easy Gluten Free Cereal,1747478,Attainable Health,NaN,PT2M,PT2M,2011-01-20T14:42:00Z,I wanted cereal one morning but didn't have an...,"""https://img.sndimg.com/food/image/upload/w_55...",...,8.0,29.4,15.3,3.8,8.8,8.4,1.0,NaN,"c(""Place walnuts in a ziploc bag then use a he...","(walnuts, vanilla yogurt, blueberries)"
165940,173710,Apple Blueberry Smoothie,37449,Sharon123,NaN,PT5M,PT5M,2006-06-19T16:30:00Z,This is a British Columbian recipe from Canada...,"c(""https://img.sndimg.com/food/image/upload/w_...",...,24.5,88.5,22.5,2.4,15.9,6.7,2.0,NaN,"""Place all ingredients in blender and blend un...","(apple, vanilla yogurt, milk, blueberries, sugar)"
418340,433731,Beautiful Berry Pie,883095,Chef mariajane,PT45M,PT0S,PT45M,2010-07-28T02:43:00Z,Make and share this Beautiful Berry Pie recipe...,character(0),...,0.0,138.1,51.4,4.2,29.9,2.9,6.0,NaN,"c(""Preheat oven to 425°F."", ""Mix sugar, flour ...","(kiwi fruits, blueberries, strawberries, granu..."
386548,400464,Yogurt Berry Parfaits,463435,Chef Buggsy Mate,PT5M,PT20M,PT25M,2009-11-20T15:34:00Z,A tasty treat when fresh berries are in season...,"c(""https://img.sndimg.com/food/image/upload/w_...",...,15.8,233.6,47.7,5.2,31.2,7.0,4.0,4 parfaits,"c(""In a small bowl, stir together the yogurt a...","(plain yogurt, prepared vanilla pudding, rolle..."
62518,66945,Frosty Fruity Smoothie,32772,Audrey M,NaN,PT5M,PT5M,2003-07-16T20:04:00Z,A cool and refreshing smoothie. To make it low...,character(0),...,31.9,119.0,69.8,4.7,57.0,9.5,2.0,NaN,"c(""Combine blueberries or huckleberries, peach...","(blueberries, huckleberries, milk, vanilla yog..."
247701,257791,The Berry Best Berries! Topping,408968,Krystalin,PT55M,PT0S,PT55M,2007-10-08T22:35:00Z,I found this recipe in a local supermarket pam...,character(0),...,0.0,2.2,13.9,2.9,9.8,0.7,NaN,NaN,"c(""In small pot, combine sugar, zest, and wate...","(sugar, lemon zest, orange, zest of, water, le..."
164419,172125,Cool Blueberry Soup,300480,dianahirkala,PT1H,PT5M,PT1H5M,2006-06-09T18:30:00Z,Make and share this Cool Blueberry Soup recipe...,character(0),...,0.0,3.1,31.9,2.9,26.7,1.3,NaN,3 cups,"c(""Combine pineapple and lemon juices, water a...","(water, lemon juice, vanilla extract, blueberr..."
245000,254992,French Blueberry Tart,133174,PaulaG,PT24H,PT20M,PT24H20M,2007-09-24T20:38:00Z,Yogurt cheese is a wonderful way to reduce cal...,"""https://img.sndimg.com/food/image/upload/w_55...",...,3.3,45.1,31.0,1.3,27.3,3.7,NaN,NaN,"c(""Place a yogurt strainer or a mesh strainer ...","(low-fat vanilla yogurt, powdered sugar, blueb..."
313051,325021,Blueberry Ice Cream,210188,Nana Lee,NaN,PT20M,PT20M,2008-09-13T13:49:00Z,Found on ricecooker site. Sounds delisc! Pos...,character(0),...,633.6,393.7,426.0,14.3,364.0,30.6,NaN,1 quart,"c(""In a 3-quart saucepan combines blueberries,...","(blueberries, sugar, vanilla extract)"


In [ ]:
recipes.columns

NameError: name 'recipes' is not defined